In [1]:
import pybryt
from lecture import pybryt_reference

# Introduction to Python

# Lecture 5

## Introduction

As we saw in [lecture 4](..\lecture4\lecture4.ipynb), Python classes combine a storing data (attributes) and functions (methods) that process that data. They act as a way to create new data types. In the previous lecture, we learned some of the individual features of classes. In this section we will learn to put multiple features together to implement and improve a class based on the functionality we need it to provide.

## Example 1: Quaternions

Just as complex numbers extend the set of real numbers into a "two-dimensional" space, [quaternions](https://en.wikipedia.org/wiki/Quaternion) extend the concept of "2D" complex numbers into a "four-dimensional" space. Quaternions are used in computer graphics, control theory, signal processing, and orbital mechanics, and more generally in areas involving applying multiple rotations or scalings using floating point arthimetic. In the realm of science, they are also used in physics to describe the quantum spin of a particle. We are going to briefly describe the mathematical properties of quaternions as numbers and then you will work to implement a class to represent them.

### Basic Definition

A quarterions is an object (viewed as a single number) consisting four real coefficients, say $a$, $b$, $c$, and $d$, along with rules for addition, mutilplication by a scalar, and multiplication by another quaternion. The rules are as follows:

1. Addition: 
  $$(a, b, c, d) + (e, f, g, h) = (a+e, b+f, c+g, d+h)$$
2. Multiplication by a scalar:
  $$r(a, b, c, d) = (ra, rb, rc, rd)$$
3. Multiplication by another quaternion: 
  $$(a, b, c, d)(e, f, g, h) = (ae - bf - cg - dh, af + be + ch - dg, ag - bh + ce + df, ah + bg - cf + de)$$
These rules are enough to define the quaternions as a number system, as well as a [vector space](https://en.wikipedia.org/wiki/Vector_space) over the real numbers.

Just as a complex number can be written in the form $a + bi$ (or in Python in a form like `1+2j`), a quaternion can be written in the form $a + bi + cj + dk$. This means they keep both engineers and mathematicians happy by having `i` & `j` both denote complex components of the number.

By the rules of multiplication above we have:
$$ i^2 = j^2 = k^2 = ijk = -1, $$
$$ ij = k, \quad jk = i, \quad ki = j. $$
In this sense, we can think of the $i$, $j$ and $k$ as three separate complex dimensions.

### Implementing Quaternions

This is enough information to start implementing a class to represent quaternions. We'll do this in a single cell immediately below. As you work through this section, we'll add more features to the class, and need to re-run the cell to update the class definition. If anything doesn't work or your results get confusing, try restarting the Python kernel and re-running that cell to see it it cleans up the problem.

#### The Class Definition cell

In [3]:
# Remember to re-run this cell after you've modified the class below
# Carry on reading to see what to implement.


class Quaternion:
    """Initializes a quaternion with the given values.

    The first value is the real part, the rest are the imaginary parts. The imaginary
    parts are i, j, and k.  For example, Quaternion(1, 2, 3, 4) represents the
    quaternion 1 + 2i + 3j + 4k.

    """

    def __init__(self, a, b, c, d):
        # Although there are no private attributes in Python, by convention, we use an
        # underscore to indicate that an attribute is private and should not be accessed
        # directly. This is a convention, and it is not enforced by the language.
        self._1 = a
        self._i = b
        self._j = c
        self._k = d

    def __str__(self):
        """Returns the string representation of the quaternion."""

        # Helper function to format components
        def format_component(value, suffix):
            return f" {'+' if value >= 0 else '-'} {abs(value)}{suffix}"

        # Format each component
        components = [
            f"{self._1}",
            format_component(self._i, "i"),
            format_component(self._j, "j"),
            format_component(self._k, "k"),
        ]

        return "".join(components)

    def __add__(self,other): 
        """ Returns a Quaternion, which is the sum of this quaternion and another, taken as an argument"""
        # Here we add the components of the two input quaternions piece-wise and output a third quaternion with components as their sums
        A = self._1 + other._1
        B = self._i + other._i
        C = self._j + other._j
        D = self._k + other._k
        return Quaternion(A,B,C,D)

    def __mul__(self,other): 
        """ Returns a Quaternion, which is the product of this quaternion and another, taken as an argument. 
        This quaternion is on the left of this multiplication -- specified because quaternion multiplication is non-commutative."""
        # Here we will implement quaternion multiplication using the relevant multiplication rule 
        # We will check if the object is a scalar or quaternion. If it is a scalar (float or int) we will convert it to a quaternion then implement the multiplication rule
        # If it is not we will raise a TypeError 

        if not isinstance(other,Quaternion): 
            if isinstance(other,int) or isinstance(other,float):
                Q1 = other
                Qi=0
                Qj=0
                Qk=0
            else: 
                raise TypeError("other not float, int, or Quaternion")

        else: 
            Q1 = other._1
            Qi = other._i
            Qj = other._j
            Qk = other._k

        # Now we have components, we can perform the quaternion multiplication
        A = self._1*Q1 - self._i*Qi - self._j*Qj - self._k*Qk
        B = self._1*Qi + self._i*Q1 + self._j*Qk - self._k*Qj
        C = self._1*Qj - self._i*Qk + self._j*Q1 + self._k*Qi
        D = self._1*Qk + self._i*Qj - self._j*Qi + self._k*Q1

        return Quaternion(A,B,C,D)


    def __rmul__(self,other): 
        """ Returns a Quaternion, which is the product of this quaternion and another, taken as an argument. 
        This quaternion is on the right of this multiplication -- specified because quaternion multiplication is non-commutative."""
        # We are going to use __mul__ within other. 
        # We know that self must be a quaternion, but other still may not be. 
        # Therefore, we check if other is a quaternion and, if not, whether it is a scalar (float or int) 
        # If it is a scalar, we turn it into a quaternion, if it is neither a scalar nor a quaternion, we return a TypeError
        if not isinstance(other,Quaternion): 
            if isinstance(other,int) or isinstance(other,float):
                other = Quaternion(other,0,0,0) # We re-assign the variable other here -- I do not know if I am allowed to do this, but I intend on finding out 
            else: 
                raise TypeError("other not float, int, or Quaternion")
        # Now that we have made sure other is a quaternion, we can use the properties of the quaternion class to calculate our answer:

        # Thinking about it further, I am not sure why we need a __rmul__ for anything that isn't a scalar. If a quaternion is on the left, then the function will be called for
        # That Quaternion and we will have no issues. 
        
        return other.__mul__(self)


    def conjugate(self): 
        """ Returns the conjugate quaterion for self as a quaternion class""" 
        # Here we just apply the conjugate for the formula.
        return Quaternion(self._1,-self._i,-self._j,-self._k)

    def __abs__(self): # Here I am assuming the third underscore below was a typo. If not, I will go back and fix it 
        """ Returns a scalar (float), equal to the absolute value of the quaternion, i.e. Q * Q.conjugate""" 
        # We can do this with quaternion multiplication, or using the metric.
        from math import sqrt # I don't know where this should go. Honestly, I am confused about where these imports should go in general because as far as I'm aware, they just need to be done once on the kernel. 
                                # This leaves functions which require on external libraries in the lurch though. Should functions call upon the libraries they need directly, or is it enough to assume that the library will be on the kernel?
        abs = sqrt(self._1**2 + self._i**2 + self._j**2 + self._k**2)
        return abs
        

    def inverse(self): 
        """ Returns the inverse of the self quaternion as a quaternion. """ 
        Q_ = self.conjugate()
        scale = self.__abs__()**-2
        
        
        inv = Q_*scale
        return inv
     

As with most classes, the first thing to do is to implement the initialisation method so that we can create new instances of the class from appropriate information. In this case, we've done this for you, and chosen the names of the attributes to be `_1`, `_i`, `_j`, and `_k`.

We've also implemented the `__str__` method, which is called when the object is printed (as well as a few other times). This is a good way to check that the initialisation method is working as expected, and we can use it to test the core class initialisation.

In [5]:
assert str(Quaternion(1, 2, 3, 4)) == "1 + 2i + 3j + 4k"

The cell block above will show no output if the class is implemented correctly. If you see a `NameError`, then make sure that you've run the cell defining the `Quaternion` class. If you see an `AssertionError`, then
make sure you haven't changed the initialisation or `__str__` method.

### Implementing the Addition Method

With the code we've already given you, you'll first need to implement the addition and multiplication methods. The addition method name to use the `+` operator is `__add__`, and it takes two arguments. The first argument is always the object itself (and as with the initialisation and `__str__` methods, it is good practice to use `self` to refer to it). The second argument is the other object being added - the Python convention is to call it `other`. This means the method signature is

```python
   def __add__(self, other):
     # method body ending with a return statement
```

Add this method to the Quaterion class above, and implement the addition rule

$$a+b\mathit{i}+c\mathit{j}+d\mathit{k} + e+f\mathit{i}+g\mathit{j}+h\mathit{k} = a+e+(b+f)\mathit{i}+(c+g)\mathit{j}+(d+h)\mathit{k}.$$

Once you're done, we can test the method by adding two quaternions together (Remember to re-run the class cell above after you've made changes to the class definition). If you see a `TypeError`, then check you've added the `__add__` method correctly (and rerun the class cell). One common mistake is geting the wrong amount of white space indentation. If you see an `AssertError` then your returned value isn't right.

In [7]:
q1 = Quaternion(1, 2, 3, 4)
q2 = Quaternion(5, 6, 7, 8)

assert str(q1 + q2) == "6 + 8i + 10j + 12k", "Got: " + str(q1 + q2)

### Implementing the Multiplication Method

Now, we want to implement multiplication. The method name is `__mul__`, and it takes the same arguments as the addition method. This means that method signature is

```python
   def __mul__(self, other):
     # method body ending with a return statement
```

Here the method body will need to be a little more complicated, since there are two possible cases:
- If `other` is a scalar number, then we need to multiply each component of the quaternion by `other`.
- If `other` is another quaternion, then we need to use the multiplication rule above.

We can check the type of `other` using the `isinstance` function. This function takes two arguments, the first is the object to check, and the second is the type to check against. It returns `True` if the object is of the given type, and `False` otherwise. For example, `isinstance(3, int)` returns `True`, and `isinstance(3, float)` returns `False`. In this case, we want to check if the `other` is a `Quaternion` object or not, using `isinstance(other, Quaternion)`.

The "quick" way to implement scalar multiplication is just to code up the multiplication rule for quaternions, and if `other` is not a `Quaternion` object, then convert it into one with the scalar as the first component, i.e. creating `Quaternion(other, 0, 0, 0)`. Alternatively, you can use a set of `if`, `elif` and (if necessary) `else` statements to handle the cases separately. In the first case, your code will look something like

```python
def __mul__(self, other):
   if not isinstance(other, Quaternion):
      other = Quaternion(other, 0, 0, 0)
  # rest of the method body    
```

in the second case, your code will look something like

```python
def __mul__(self, other):
   if isinstance(other, Quaternion):
      # result = multiplication rule for quaternion
      return result
   else:
      # multiplication rule for scalar
      return result
```
 
Try to implement all this, and then run our test cases below.

In [9]:
q1 = Quaternion(1, 2, 3, 4)
q2 = Quaternion(5, 6, 7, 8)

assert str(q1 * q2) == "-60 + 12i + 30j + 24k", "Got: " + str(q1 * q2)
assert str(q2 * q1) == "-60 + 20i + 14j + 32k", "Got: " + str(q2 * q1)
assert str(q1 * 3) == "3 + 6i + 9j + 12k", "Got: " + str(q1 * 3)

### Additional Methods

We've now implemented a method to multiply "from the left", i.e. solving `q * other`, but for the purposes of scalar multiplication, we also need to implement a method to multiply "from the right", i.e. solving `other * q`.

The appropriate special method (also called a "magic method" or "dunder method") name is `__rmul__`, and it takes the same arguments as the multiplication method, except this time "other" is the object on the left. This means that method signature is

```python
   def __rmul__(self, other):
     # method body ending with a return statement
```

Note that we can't just call `return self.__mul__(other)` here, since for quaternions, multiplication is not commutative (i.e. `q1 * q2` is not equal to `q2 * q2` for general quaternions `q1` and `q2`). Again there are several ways to solve this, but the one involving least code is to make `other` into a `Quaternion` object if it isn't already, and then call its `__mul__` method on self (either explicity as `other.__mul__(self)` or implicitly through `other*self`).



In [11]:
q1 = Quaternion(1, 2, 3, 4)

assert str(3 * q1) == "3 + 6i + 9j + 12k", "Got: " + str(3 * q1)

We've now implemented the core arthmetic operations for quaternions, and can move on to more advanced features. Let's start by defining the conjugate of a quaternion. Just as the conjugate of a complex number $z= a+b\mathit{i}$ is $\bar{z} = a - b\mathit{i}$,  the conjugate of a quaternion $q = a + b\mathit{i} + c\mathit{j} + d\mathit{k}$ is defined as $\bar{q} = a - b\mathit{i} - c\mathit{j} - \mathit{k}$. 

We can implement this as a method `conjugate` that takes no additional arguments, and returns a new quaternion object with the "imaginary" parts negated. The method signature should be

```python
   def conjugate(self):
     # method body ending with a return statement
```

Note that $q\bar{q} := \|q\|^2 = a^2 + b^2 + c^2 + d^2$, which is a positive real number. We can define the inverse or reciprocal of a quaternion as $\frac{\bar{q}}{\|q\|^2}$, and implement this as a method `inverse`.
```python
   def inverse(self):
     # method body ending with a return statement
```
It's not technically correct to call this division, due to the non-commutative nature of quaternion multiplication. This means that there are two "inverse functions" for multiplication, one which solves $xa = b$ for $x$ and one which solves $ax = b$. Using the notation $a^{-1}$ for the inverse above, the inverse from the right is $ba^{-1}$ and the inverse is $a^{-1}b$.

Instead of computing the magnitude $|q|$ inside the `inverse` method, implement the special `__abs__` method. This method will be called when built in `abs` function is called on a Quaternion object. Its signature is:
```python
   def __abs____(self):
     # method body ending with a return statement
```

Try to implement the `conjugate` and `inverse` methods in your `Quaternion` class above, and then run the test cases below.

In [13]:
q1 = Quaternion(1, 1, 1, 1)

assert str(q1.conjugate()) == "1 - 1i - 1j - 1k", "Got: " + str(q1.conjugate())
assert str(abs(q1)) == "2.0", "Got: " + str(abs(q1))
assert str(q1.inverse()) == "0.25 - 0.25i - 0.25j - 0.25k", "Got: " + str(q1.inverse())

### Refactoring the class (optional)

We've now implemented all the basic functionality of quaternions. However, the class is not particularly user-friendly. Here is a list of some things you could improve:

- The `__init__` method is currently not very user-friendly. It would be nice if we could create a quaternion in a few extra ways:
  - Let `Quaternion(1) == Quaternion(1, 0, 0, 0)`.
  - Let `Quaternion(1+2j) == Quaternion(1, 2, 0, 0)`.
  - Let `Quaternion([1, 2, 3, 4]) == Quaternion(1, 2, 3, 4)`.
  
This can be done in a number of ways. One lower effort solution is to use the `*args` and/or `**kwargs` syntax to allow for a variable number of arguments and keyword arguments. 

In [15]:
def function_with_var_args(*args, **kwargs):
    print(args, kwargs)


function_with_var_args(1, 2)
function_with_var_args(1, 2, 3, 4, 5)
function_with_var_args(1, 2, 3, 4, 5, a=6, b=7, c=8)

(1, 2) {}
(1, 2, 3, 4, 5) {}
(1, 2, 3, 4, 5) {'a': 6, 'b': 7, 'c': 8}


Another things we can implement are:

- We have implemented the `__str__` method, but not the `__repr__` method. The `__repr__` method is called when the object is printed in a list or dictionary, and should return a string that can be used to recreate the object. For example, `repr(Quaternion(1, 2, 3, 4))` should return the string `"Quaternion(1, 2, 3, 4)"`. If we copy/paste the output of `__repr__`, we should get the same object.
- We have implemented the `__add__`, `__mul__` and  `__rmul__`, methods, but not the `__sub__`, `__eq__` methods (or a few others).


In [17]:
assert repr(Quaternion(1, -2, 3, 4)) == "Quaternion(1, -2, 3, 4)"

assert Quaternion(1) == Quaternion(1, 0, 0, 0)
assert Quaternion(1, 2) == Quaternion(1, 2, 0, 0)
assert Quaternion(b=2) == Quaternion(0, 2, 0, 0)

assert Quaternion(1, 2, 3, 4) - Quaternion(5, 6, 7, 8) == Quaternion(-4, -4, -4, -4)

AssertionError: 

In [ ]:
with pybryt.check(pybryt_reference(5, 1)):
    q1 = Quaternion(1, -2, 3, -4)
    q2 = Quaternion(15, 2, 0, -1)

    repr(q1), str(q1)
    repr(q1 + q2)
    repr(q2 - q1)
    repr(5 * q1)
    repr(q1 * q2)
    abs(q1)
    repr(q1.conjugate())

## Example 2: Building Triangles

### Problem Statement

The second example we will consider is building a collection of classes to represent the information present in triangles. We've defined a class for points in two dimensions. You'll then use this to define a class for line segments (defined by two points), and then a class for triangles (defined by three points). We'll then add methods to calculate the area and perimeter of the triangle, and the length of the line segment.

### The Point Class Defintion

 We'll define a class to represent a point in 2D space. You will then need to implement a line segment (to represent an edge for a triangle), and finally a triangle. We will add appropriate methods to each class to calculate the area and perimeter of a triangle and the length of a line segment.

In [26]:
from dataclasses import dataclass


@dataclass
class Point:
    x: float
    y: float

Here we've used the `dataclass` decorator from the `dataclasses` module to define the class. This is a more concise way to define a class, and is particularly useful when the class is just a container for data.

The `dataclass` decorator automatically adds an `__init__` method, and `__repr__` method, and `__eq__` method, and a few other things. The `__init__` method takes the same arguments as the class, and the `__repr__` method returns a string that can be used to recreate the object. The `__eq__` method checks if two objects are equal. 

In [28]:
# Examples of usage
p1 = Point(1.0, 2.0)
p2 = Point(x=2.5, y=4.0)

print(p1, p2)
print(p1 == p2)
print(p1 == Point(1.0, 2.0))

Point(x=1.0, y=2.0) Point(x=2.5, y=4.0)
False
True


### The LineSegment Class Implementation

Given the `Point` class definition above, we can now implement a `LineSegment` class which takes two `Point` objects as arguments to its `__init__` method. The `__init__` method should store these two points as attributes of the object. We'll also want a `length` method to calculate the length of the line segment.

In [30]:
class LineSegment:
    def __init__(self, start: Point, end: Point):
        """ Initialises the line segment with given start and end points""" 
        if not (isinstance(start,Point) and isinstance(end,Point)):
            raise TypeError("Input values are not Points")
        # Write appropriate code here to store the start and end points
        self.start = start
        self.end = end 
        

    def __repr__(self):
        """ Returns a string version of the line segment, which can be used to rebuild it.""" 
        return f"LineSegment({self.start}, {self.end})"

    def length(self) -> float:
        """ returns the length of the line segment as a float, using Pythagoras' theorem"""
        # Write appropriate code here, updating the class to
        # return the length of the line segment
        from math import sqrt
        L = sqrt((self.end.x - self.start.x)**2 + (self.end.y - self.start.y)**2)

        return L

    def midpoint(self):
        """ Finds the midpoint of the line segment and outputs it as a Point object """ 
        # We will do this by taking the means of the coordinates of the start and end points: 
        x_mean = (self.start.x + self.end.x)*0.5
        y_mean = (self.start.y + self.end.y)*0.5
        return Point(x_mean,y_mean)
    
    
    def intersects(self,other): 
        """ returns a boolean corresponding to whether the two line segments intersects. Other must be a line segment"""
        # We insist other must be a line segment and raise an error if it is not 
        if not isinstance(other,LineSegment): 
            raise TypeError("Argument must be LineSegment")

        # We will take the approach outlined below and use simultaneous parametric equations to solve for parameters t1 and t2
        # Then we will impose the condition that t1 and t2 must be between 0 and 1 for an interection to have occurred 

        # Let us unpack the relevant parameters 
        px = self.start.x
        dpx = self.end.x - px
        py = self.start.y
        dpy = self.end.y - py
        
        qx = other.start.x
        dqx = other.end.x - qx
        qy = other.start.y
        dqy = other.end.y - qy

        # We write the simultaneous parametric equations as a matrix equation which has a solution if the matrix M= [dpx,-dqx;dpy,-dqy] is invertible
        # Implementing this condition
        detM = (-dpx*dqy + dpy*dqx)
        
        if detM == 0: 
            return False # no solutions for t1,t2

        else: 
            t1 = detM**-1 * (dqy*(px-qx) + dqx*(qy-py))
            t2 = detM**-1 * (dpy*(px-qx) + dpx*(qy-py))

            if t1 >= 0 and t1 <= 1 and t2 >= 0 and t2 <= 1: 
                return True 
            else: 
                return False 
            
        

Here we've used the optional "type hints" feature of Python to specify the types of the arguments to the `__init__` method. This is not necessary (nor it is checked), but can be useful for documentation and for catching errors. The type hints are specified in the method signature, and are separated from the argument name by a colon. For example, `p1: Point` specifies that `p1` is a `Point` object. Similarly, `-> float` suggests that the `length` method returns a `float` object. Once again, this is not checked anywhere, and only provides a hint to the user, either in code or through documentation.

We'll also need some tests on the class:

In [32]:
from math import isclose

p1 = Point(0, 0)
p2 = Point(3, 4)

assert isclose(LineSegment(p1, p2).length(), 5)

#### Extending the LineSegment Class


- add a method to the `LineSegment` class called `midpoint()`. It should return a `Point` object representing the midpoint of the line segment.
- (more challenging) add a method to the `LineSegment` class called `intersects`. It should take another `LineSegment` object as an argument, and return `True` if the two line segments intersect, and `False` otherwise. You can assume that the line segments are not colinear if you'd like to reduce your workload.
  As a hint, you can use the following algorithm to check if two line segments intersect:
    - We can write any line segment as a parametric equation, i.e. $\mathbf{r}(t) = \mathbf{p}_1 + t(\mathbf{p}_2 - \mathbf{p}_1)$, where $\mathbf{p}_1$ and $\mathbf{p}_2$ are beginning and end of the line segment, and $t$ is a real number between 0 and 1.
    - Two line segments intersect if and only if the parametric equations of the two line segments have a common solution, i.e. there exists $t_1$ and $t_2$ such that $\mathbf{r}_1(t_1) = \mathbf{r}_2(t_2)$.
    - Substituting the parametric equations into the equation $\mathbf{r}_1(t_1) = \mathbf{r}_2(t_2)$, we get a system of 2 linear equations in $t_1$ and $t_2$. If the system has a solution (and the values of $t_1$ and $t_2$ are in the range $0-1$), then the line segments intersect.

In [34]:
p1 = Point(-2, -2)
p2 = Point(2, 2)

assert LineSegment(p1, p2).midpoint() == Point(0, 0)

line_segment1 = LineSegment(Point(0, 0), Point(10, 10))
line_segment2 = LineSegment(Point(1, 1), Point(11, 11))
line_segment3 = LineSegment(Point(10, 0), Point(0, 10))
line_segment4 = LineSegment(Point(0, 0), Point(10, 10))

assert not line_segment1.intersects(line_segment2)
assert line_segment1.intersects(line_segment3)
assert line_segment3.intersects(line_segment4)

In [35]:
with pybryt.check(pybryt_reference(5, 2)):
    l1 = LineSegment(Point(0, 0), Point(1, 1))
    l2 = LineSegment(Point(0, 1), Point(1, 0))

    repr(l1)
    l1.length()
    l2.midpoint().x
    l2.midpoint().y
    l1.intersects(l2)

REFERENCE: exercise-5_2
SATISFIED: False
MESSAGES:
  - SUCCESS: Your repr method is correctly implemented. Well done!
  - SUCCESS: Your length method is correct.
  - SUCCESS: Your midpoint calculation (x) is correct.
  - SUCCESS: Your midpoint calculation (y) is correct.
  - ERROR: Your intersection calculation seems to be wrong.


In [36]:
print(l1.intersects(l2)) # It seems to work correctly 

True


### The Triangle class Implementation

Finally, we need to implement a `Triangle` class. We'll choose to use an initialisation method that takes three `Point` objects as arguments, and stores them as attributes of the object. We'll also want methods to calculate the area and perimeter of the triangle.

In [38]:
import numpy as np 
import math

class Triangle:
    def __init__(self, a: Point, b: Point, c: Point):
        
        """ Initialises a Triangle object, defined by three vertex points listed in order. Points must be of the Point class
        Note that Triangle(a,b,c) is a different object to Triangle(b,a,c), even though they would be plotted as the same Triangle."""
        if not (isinstance(a,Point) and isinstance(b,Point) and isinstance(c,Point)):
            raise TypeError("Input values are not Points")
        
        self.A = a
        self.B = b
        self.C = c
        

    def area(self) -> float:
        """Returns the area of the Triangle as a float."""
        (AB,BC,CA) = self.edges()
        (alpha,beta,gamma) = self.angles()

        Area = 0.5 * AB.length() * CA.length() * np.sin(alpha)
        return Area 
        

    def perimeter(self) -> float:
        """Returns the perimeter of the Triangle as a float."""
        (AB,BC,CA) = self.edges()
        perimeter = AB.length() + BC.length() + CA.length()
        
        return perimeter
        
    
    def edges(self): 
        """Returns a 3-D tuple, which contains each of the Triangle's sides as LineSegment objects
        The first LineSegment will correspond to the line AB, the second BC, and the third CA"""
        AB = LineSegment(self.A,self.B)
        BC = LineSegment(self.B,self.C)
        CA = LineSegment(self.C,self.A) 

        return (AB,BC,CA)

    def angles(self): 
        """Returns a 3-D tuple, which contains each of the triangle's interior angles in radians
        The first value of the tuple corresponds to the first vertex of the triangle (a)"""

        # We will calculate the angles using the cosine formula and numpy's arccos function
        (AB,BC,CA) = self.edges()

        # What follows is labelling convention in geometry where the length of the side opposite point A is a 
        c = AB.length()
        a = BC.length()
        b = CA.length()
        
        angle_A = np.arccos((b**2 + c**2 - a**2)/(2*b*c))
        angle_B = np.arccos((a**2 + c**2 - b**2)/(2*a*c))
        angle_C = np.arccos((b**2 + a**2 - c**2)/(2*b*a)) # np.arccos outputs in radians 

        return (angle_A,angle_B,angle_C)


    def contains_point(self,p: Point):
        "Returns a Boolean, which is True when point p lies within the triangle." 

        # We continue to use the convention that a is the side opposite point A  
        # Therefore normal_a is the normal to that side 
        # We will pair up normal_a with line Bp and take their dot products. normal_b with Cp etc. 
        
        check_BC = (p.x - self.B.x)*(self.C.y - self.B.y) + (p.y - self.B.y)*(self.B.x - self.C.x)
        check_CA = (p.x - self.C.x)*(self.A.y - self.C.y) + (p.y - self.C.y)*(self.C.x - self.A.x)
        check_AB = (p.x - self.A.x)*(self.B.y - self.A.y) + (p.y - self.A.y)*(self.A.x - self.B.x)

        print(check_BC, check_CA, check_AB)

        # If all of these are positive, the triangle lies inside the circle. If not, it does not
        if check_BC >= 0 and check_CA >= 0 and check_AB >= 0: 
            return True 
        elif check_BC <= 0 and check_CA <= 0 and check_AB <= 0: # This was added later 
            return True 
        else: 
            return False
        

        

# The tests do not seem to align with what we have been asked to do 
p1 = Point(0, 0)
p2 = Point(1, 0)
p3 = Point(0, 1)

t = Triangle(p1, p2, p3)

assert isclose(t.area(), 0.5), "Check your area calculation"
assert isclose(t.perimeter(), 1 + 1 + math.sqrt(2)), "Check your perimeter calculation"
#assert isclose(t.edges()[0], 1) # I received a typeError here, saying that the result should be a real number. I disagree. The values of the tuple are line segments corresponsing to the sides

#assert isclose(t.edges()[1], math.sqrt(2))
#assert isclose(t.edges()[2], 1)
assert isclose(t.angles()[0], math.pi / 4)
assert isclose(t.angles()[1], math.pi / 2)
assert isclose(t.angles()[2], math.pi / 4)
assert not t.contains_point(Point(1, 1))

In [40]:
p1 = Point(0, 0)
p2 = Point(1, 0)
p3 = Point(0, 1)

t = Triangle(p1, p2, p3)

print(t.angles()[0],t.angles()[1],t.angles()[2])
t.contains_point(Point(1, 1))

# Okay, so, I've figured it out. the test we were given is wrong. All dot products are EITHER positive or negative -- depending on how you choose to go around the triangle
# a line does not have a directionally defined normal, therefore we have a degree of freedom in our sign. 

1.5707963267948968 0.7853981633974481 0.7853981633974481
1 -1 -1


False

### Extending the Triangle Class

- add a new method to the `Triangle` class, `edges()`, that returns a `tuple` containing the three `LineSegment`s which form the edges of the `Triangle`. 
The edges should be ordered so that the first vertex of the first edge is the first vertex of the triangle, and the second vertex of the first edge is the second vertex of the triangle, as well as being the first vertex of the second edge, and so on.
- add a method to the `Triangle` class called  `angles()`. It should return a tuple of three floats, each representing the interior angle _in radians_ of the vertices of the triangle, ordered so that the first angle is at the first vertex of the triangle, and so on.
- (more challenging) add a method to the `Triangle` class called `contains_point` that takes a `Point` object as an argument, and returns `True` if the point is inside (or on an edge of) the triangle, and `False` otherwise. You can use the following algorithm to solve this problem:
  - For each edge of the triangle, calculate the normal vector to the edge.   you can do this by calculating
    $$\mathbf{n} = \begin{pmatrix} y_2 - y_1 \\ x_1 - x_2 \end{pmatrix},$$
    where $(x_1, y_1)$ and $(x_2, y_2)$ are the coordinates of the two vertices of the edge.
  - Calculate the vector from one of the vertices of the edge to the point.
  - If the dot product of these two vectors is positive for all edges, then the point is inside the triangle. If the dot product is negative for any edge, then the point is outside the triangle. If the dot product is zero for an edge, (and positive for the others) then the point is on the edge of the triangle.
  
  You can use additional packages such as  `numpy`if you like, but it is not necessary.


# Testing here is also strange, and I would like to talk this over 
p1 = Point(0, 0)
p2 = Point(1, 0)
p3 = Point(0, 1)

t = Triangle(p1, p2, p3)

assert isclose(t.edges()[0], 1)
assert isclose(t.edges()[1], math.sqrt(2))
assert isclose(t.edges()[2], 1)
assert isclose(t.angles()[0], math.pi / 4)
assert isclose(t.angles()[1], math.pi / 2)
assert isclose(t.angles()[2], math.pi / 4)
assert not t.contains_point(Point(1, 1))

### Extending the shape design via inheritance (optional)

- We can also extend functionality to allow for more shapes, such as squares, hexagons and so on. The "best" way to do this in Python is to define a new "abstract" class called `Polygon` that implements reusable methods for generic shapes (where possible) in terms of the Points or edges of the polygon, and then define a new class for each shape that inherits from `Polygon` as a base class and which implements the methods that are specific to that shape (for example the `area` method for a triangle is different to the `area` method for a square).

By defining `Polygon` as 

In [44]:
from abc import ABC, abstractmethod


class Polygon(ABC):
    def __init__(self, vertices):
        # Write appropriate code here, accepting a collection
        # of Points as arguments, a representing a vertex of the polygon.
        pass

    def perimeter(self):
        # Write appropriate code here, returning the perimeter of the polygon
        # based on the stored vertices.
        pass

    def edges(self):
        # Write appropriate code here, returning a tuple of LineSegments
        # representing the edges of the polygon
        pass

    @abstractmethod
    def area(self):
        # Write appropriate code here, returning the area of the polygon
        raise NotImplementedError

In [45]:
class Quadrilateral(Polygon):
    def __init__(self, a, b, c, d):
        # Here, super() will call the __init__ method of the parent class -
        # Polygon.__init__().
        super().__init__(a, b, c, d)

    def area(self):
        # Write appropriate code here, returning the area of the quadrilateral.
        raise NotImplementedError

## Example 3: Designing your own class

### Problem Statement

The final problem is always the most challenging: designing your own class from scratch. We'll give you a problem statement, and you'll need to design a class to solve it. The problem statement is as follows:

Build a class to represent the data provided from a [weather station](https://en.wikipedia.org/wiki/Weather_station). The station has the following fixed data attached to it:
- A name (a string)
- A location (a tuple of two floats, representing the latitude and longitude)
- A height above sea level (a float, representing the height in meters)

The station also has (at a minimum) the following data that change over time:

- A temperature (a float, representing the temperature in degrees Celsius)
- A humidity (a float, representing the humidity as a percentage)
- The pressure (a float, representing the pressure in hPa)

The class should have methods to do the following:

- An initialisation method (`__init__`) that takes the name, location, and height above sea level as arguments, and stores them, as well as doing some initialisation of the temperature, humidity and pressure (plus other variables if you've added them).
- A method to update the temperature, humidity, and pressure.
- Appropriately implemented `__str__` and `__repr__` methods.
- Methods to calculate appropriate "weather" metrics, such as the [dew point](https://en.wikipedia.org/wiki/Dew_point) (it's ok to use the simplified formula), the [heat index](https://en.wikipedia.org/wiki/Heat_index), and so on, and to convert between different temperature scales (e.g. Celsius, Fahrenheit, Kelvin). You may want to make use of the `@property` decorator to make these methods feel like attributes to the user.

In [47]:
import pendulum as pnd # So we can store some time data 

# We will support the following units for temperature: Celsius, Kelvin, Fahrenheit
# We will support the following units for pressure: bar, Pascals, mmHg, psi
# We will assume equivalent mass units are used for humidity, and that it is stored as a %. 

class Weather_Station: 
    def __init__(self,name,location: tuple,elevation):
        """Initialises an object in the Weather_Station class.
            Temperature Data is stored in Celsius and output in different units on request
            Pressure Data is stored in bar and output in different units on request.
            The times at which each of the variables were last updated are stored as pendulum datetime objects. 
            """
        self._name = name
        self._latitude = location[0]
        self._longitude = location[1]
        self._location = location
        self._elevation = elevation
        self._temperature = None
        self._temperature_update_time = None
        self._pressure = None
        self._pressure_update_time = None
        self._humidity = None 


    def __str__(self):
        return f"{self._name}"
        

    def __repr__(self):
        return f"Weather_Station(Name: {self._name}, Location: {self._location}, Elevation: {self._elevation})"
        

    def update_T(self,temperature,unit="C"): 
        if unit == "C":
            self._temperature = temperature
        elif unit == "F":
            self._temperature = (temperature - 32)/1.8 # This connverts the input to Celsius and saves it in the object
        elif unit == "K":
            self._temperature = temperature + 273.15 # This connverts the input to Celsius and saves it in the object
        else: 
            raise ValueError("Temperature unit not supported.") # If we don't recognise the string input for unit, we return an error

        self._temperature_update_time = pnd.now('UTC')
        

    def update_P(self,pressure,unit="bar"):
        if unit == "bar":
            self._pressure = pressure
        elif unit == "Pa": 
            self._pressure = pressure/ 100000 # This converts to bar and saves it in the object 
        elif unit == "mmHg":
            self._pressure = pressure/750.06 # This converts to bar and saves it in the object 
        elif unit == "psi":
            self._pressure = pressure/14.5 # This converts to bar and saves it in the object 
        else:
            raise ValueError("Pressure unit not supported.") 
        
        self._pressure_update_time = pnd.now('UTC')
            

    def update_H(self,humidity):
        # No need to handle unit conversions here
        self._humidity = humidity
        self._humidity_update_time = pnd.now('UTC')


    def get_T(self,unit="C"):
        if unit == "C": 
            return self._temperature
        elif unit == "F": 
            return self._temperature * 1.8 + 32
        elif unit == "K":
            return self._temperature + 273.15
        else: 
            raise ValueError("Temperature unit not supported.")
            

    def get_P(self,unit="bar"): 
        if unit == "bar": 
            return self._pressure 
        elif unit == "Pa":
            return self._pressure * 100000
        elif unit == "mmHg": 
            return self._pressure * 750.06
        elif unit == "psi": 
            return self._pressure * 14.5
        else: 
            raise valueError("Pressure unit not supported.") 
            

    def get_H(self): 
        return self._humidity
        

    def get_time(self,target): 
        """Target can take values "T", "P" or "H" for temperature, pressure, and humidity respectively."""
        if target == "T": 
            return self._temperature_update_time
        elif target == "P": 
            return self._temperature_update_time
        elif target == "H":
            return self._temperature_update_time 
        else: 
            raise ValueError("Invalid target")


    def dew_point(self):
        dp = self._temperature - (100-self._humidity)/5
        return dp
        

    def heat_index(self): 
        c = [-8.785,1.611,2.339,-0.1461,-0.01231,-0.01642,2.212e-3,7.255e-4,-3.582e-6]
        hi =  c[0] + c[1]*self._temperature + c[2]*self._humidity + c[3]*self._temperature*self._humidity + c[4]*self._temperature**2 + c[5]*self._humidity**2 + c[6]*self._temperature**2*self._humidity + c[7]*self._temperature*self._humidity**2 + c[8]*self._temperature**2*self._humidity**2
        return hi

In [48]:
print(pnd.now('UTC'))

type(('UTC',1))

2025-10-03 10:05:32.764326+00:00


tuple

In [49]:
# Let's test if the string functions work
Station1 = Weather_Station('Station 1',(0,0),0)
print(str(Station1))
print(repr(Station1))

# Let's check if storing and recalling data works 
Station1.update_T(100,unit="F")
Station1.update_P(1)
Station1.update_H(60)

print(Station1.get_T(),Station1.get_P("mmHg"),Station1.get_P("psi"),Station1.get_time("T"))

# Let's see if changes are made correctly
Station1.update_T(36)

print(Station1.get_T(),Station1.get_time("T")) # It has changed correctly

# Now, lets check our functions
print("The dew point for Station1 is:", Station1.dew_point())
print("The Heat index for Station1 is:", Station1.heat_index())
# Both of these compare favourably to online examples. 
# I think it's safe to say we have all the functinality we need.

Station 1
Weather_Station(Name: Station 1, Location: (0, 0), Elevation: 0)
37.77777777777778 750.06 14.5 2025-10-03 10:05:32.769579+00:00
36 2025-10-03 10:05:32.769681+00:00
The dew point for Station1 is: 28.0
The Heat index for Station1 is: 48.2269808


In [51]:
# Let's check that the class above does everything it is supposed to do

Station1 = Weather_Station('station 1', (0,0), 0)

# Let's check __str__
print(Station1)

#Let's check __repr__
Station1.__repr__

# Now, let's add a new temperature
Station1.update_T(30)
print(Station1._temperature)

# Let's add two more, one in Fahrenheit and one in a made-up unit 'J', to see what happens 
Station1.update_T(25,unit="F")
try:
    Station1.update_T(26,unit="J")
except ValueError:
    pass

print(Station1._temperature)

# Now, let us test out our T function, 
print(Station1.get_T())
print(Station1.get_T(unit='K'))

station 1
30
-3.888888888888889
-3.888888888888889
269.26111111111106
